## Project description <br> 

Classification is probably the most popular task that you would deal with in real life. Text in the form of blogs, posts, articles, etc. is written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to create a classifier that predicts multiple features of the author of a given text.We have designed it as a Multilabel classification problem.

# Dataset

Blog Authorship Corpus Over 600,000 posts from more than 19 thousand bloggers The Blog Authorship Corpus consists of the collected posts of 19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million words - or approximately 35 posts and 7250 words per person. Each blog is presented as a separate file, the name of which indicates a blogger id# and the blogger’s self-provided gender, age, industry, and astrological sign. (All are labeled for gender and age but for many, industry and/or sign is marked as unknown.) All bloggers included in the corpus fall into one of three age groups: 
*   8240 "10s" blogs (ages 13-17), 
*   8086 "20s" blogs(ages 23-27) 
*   2994 "30s" blogs (ages 33-47)

For each age group, there is an equal number of male and female bloggers. Each blog in the corpus includes at least 200 occurrences of common English words. All formatting has been stripped with two exceptions. Individual posts within a single blogger are separated by the date of the following post and links within a post are denoted by the label urllink.

Link to dataset: https://www.kaggle.com/rtatman/blog-authorship-corpus/downloads/blog-authorship-corpus.zip/2at

# Approach and Steps


1. Load the dataset (5 points)
a. Tip: As the dataset is large, use fewer rows. Check what is working well on your machine and decide accordingly.
2. Preprocess rows of the “text” column (7.5 points)
a. Remove unwanted characters
b. Convert text to lowercase
c. Remove unwanted spaces
d. Remove stopwords
3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points)
a. Label columns to merge: “gender”, “age”, “topic”, “sign”
b. After completing the previous step, there should be only two columns in your data frame i.e. “text” and “labels” as shown in the below image

![image#1](https://drive.google.com/file/d/1eVPo-opGSk5xXJRF_XIcnIry9rCHO_Co/view?usp=sharing)

4. Separate features and labels, and split the data into training and testing (5 points)
5. Vectorize the features (5 points)
a. Create a Bag of Words using count vectorizer
i. Use ngram_range=(1, 2)
ii. Vectorize training and testing features
b. Print the term-document matrix
6. Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. Check below image for reference (5 points)

![image#2](https://drive.google.com/file/d/13AMyhdMqkxIZI9DTihhXCxwpX35O9apk/view?usp=sharing)

7. Transform the labels - (7.5 points) As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn
a. Convert your train and test labels using MultiLabelBinarizer
8. Choose a classifier - (5 points) In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large.
a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label
b. As One-vs-Rest approach might not have been discussed in the sessions, we are providing you the code for that

![image#3](https://drive.google.com/file/d/1YjWRl2lxCPBUS2lj5Mri94l8JcGntowD/view?usp=sharing)

9. Fit the classifier, make predictions and get the accuracy (5 points)
a. Print the following
i. Accuracy score
ii. F1 score
iii. Average precision score
iv. Average recall score
v. Tip: Make sure you are familiar with all of them. How would you expect the things to work for the multi-label scenario? Read about micro/macro/weighted averaging
10. Print true label and predicted label for any five examples (7.5 points)

### 1. a  Load the dataset

In [72]:
import pandas as pd
import numpy as np
# Reading data from file
data = pd.read_csv('blogtext.csv').dropna()

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681284 entries, 0 to 681283
Data columns (total 7 columns):
id        681284 non-null int64
gender    681284 non-null object
age       681284 non-null int64
topic     681284 non-null object
sign      681284 non-null object
date      681284 non-null object
text      681284 non-null object
dtypes: int64(2), object(5)
memory usage: 41.6+ MB


In [74]:
data.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


###   1.b limiting dataframe to 10000 rows 

In [75]:
df=data.iloc[:10000,0:]

In [76]:
df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
9995,1705136,female,25,indUnk,Pisces,"19,May,2004",take me home with you forever where I ...
9996,1705136,female,25,indUnk,Pisces,"23,June,2004",seductive secretness behind doors warn...
9997,1705136,female,25,indUnk,Pisces,"21,June,2004",For being so kind to me when I need yo...
9998,1705136,female,25,indUnk,Pisces,"09,June,2004",blurry outside sounds as people mingle...


### 2. Preprocess NLP text (lowercase, remove special characters, remove numbers, remove emails, etc)

Defined a function preprocess, this will cater to lower case conversion, removal of html tags, url, stemming and lemmatization of the words.

In [77]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yogeshk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yogeshk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['text']=df['text'].map(lambda s:preprocess(s))

In [ ]:
df

In [9]:
df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

### 3. Create label column to merge: “gender”, “age”, “topic”, “sign”



In [10]:
df['label']=df['gender'].astype(str)+','+df['age'].astype(str)+','+df['topic'].astype(str)+','+df['sign'].astype(str)

In [11]:
df

,id,gender,age,topic,sign,date,text,label
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages pdf files wait untill team le...,"male,15,Student,Leo"
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie aarde maak eigen water...,"male,15,Student,Leo"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"male,15,Student,Leo"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"
...,...,...,...,...,...,...,...,...
1995,589736,male,35,Technology,Aries,"05,August,2004",mir lmao funny,"male,35,Technology,Aries"
1996,589736,male,35,Technology,Aries,"05,August,2004",read section article perhaps rest article keep...,"male,35,Technology,Aries"
1997,589736,male,35,Technology,Aries,"05,August,2004",guess could asked victoria secret fashion show...,"male,35,Technology,Aries"
1998,589736,male,35,Technology,Aries,"05,August,2004",gamers urllink live silicon,"male,35,Technology,Aries"


### Taking datasets with only text and label

In [12]:
df1 = df.iloc[:,6:8]

In [13]:
df1

,text,label
0,info found pages pdf files wait untill team le...,"male,15,Student,Leo"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,het kader van kernfusie aarde maak eigen water...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"
...,...,...
1995,mir lmao funny,"male,35,Technology,Aries"
1996,read section article perhaps rest article keep...,"male,35,Technology,Aries"
1997,guess could asked victoria secret fashion show...,"male,35,Technology,Aries"
1998,gamers urllink live silicon,"male,35,Technology,Aries"


### 4. Separate features and labels, and split the data into training and testing (5 points)



In [16]:
from sklearn.model_selection import train_test_split

x = df1['text']
y = df1['label']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1)

In [17]:
print(x_train.shape)
print(x_test.shape)

(1400,)
(600,)


In [18]:
print(y_train.shape)
print(y_test.shape)

(1400,)
(600,)


In [19]:
y_train

1194                [male, 25, Internet, Aries]
45      [male, 33, InvestmentBanking, Aquarius]
1477              [male, 35, Technology, Aries]
1293               [male, 39, Education, Virgo]
1736              [male, 35, Technology, Aries]
                         ...                   
1791              [male, 35, Technology, Aries]
1096               [female, 15, Student, Libra]
1932              [male, 35, Technology, Aries]
235               [male, 15, Student, Aquarius]
1061               [female, 15, Student, Libra]
Name: labels, Length: 1400, dtype: object

# 5. Vectorize the features

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 2))

# create document-term matrices
x_train_ct = cv.fit_transform(x_train)
x_test_ct = cv.transform(x_test)

In [21]:
#checking the vocabulary size
len(cv.vocabulary_)

138289

In [22]:
#what is there in vocabulary
cv.vocabulary_

{'decided': 28244,
 'document': 31085,
 'little': 68275,
 'adventure': 1438,
 'america': 3673,
 'previous': 92372,
 'attempt': 7234,
 'blog': 12212,
 'urllink': 127370,
 'fallen': 39098,
 'wayside': 131442,
 'somewhat': 110251,
 'think': 120102,
 'blogger': 12349,
 'extra': 38285,
 'features': 39943,
 'able': 95,
 'rectify': 97192,
 'situation': 108262,
 'besides': 10828,
 'give': 47214,
 'somewhere': 110270,
 'moan': 76151,
 'red': 97201,
 'tape': 117759,
 'infested': 58240,
 'wold': 134446,
 'end': 34627,
 'immigrate': 57455,
 'days': 27728,
 'anyways': 5151,
 'blogging': 12382,
 'nearly': 79418,
 'weeks': 132043,
 'country': 24822,
 'finally': 41016,
 'new': 80258,
 'place': 89108,
 'live': 68538,
 'right': 99412,
 'next': 80574,
 'reston': 98850,
 'town': 124104,
 'center': 18056,
 'rent': 98129,
 'reasonable': 96798,
 'big': 11304,
 'bonus': 12983,
 'within': 134196,
 'walking': 129918,
 'distance': 30858,
 'job': 60453,
 'good': 48438,
 'given': 47331,
 'hard': 52239,
 'get': 461

Building DTM and vectorizing train and test features

In [23]:
x_train_ct.shape

(1400, 138289)

In [24]:
print(x_train_ct[0])

  (0, 28244)	1
  (0, 31085)	1
  (0, 68275)	2
  (0, 1438)	1
  (0, 3673)	1
  (0, 92372)	1
  (0, 7234)	2
  (0, 12212)	1
  (0, 127370)	2
  (0, 39098)	1
  (0, 131442)	1
  (0, 110251)	1
  (0, 120102)	2
  (0, 12349)	1
  (0, 38285)	1
  (0, 39943)	1
  (0, 95)	1
  (0, 97192)	1
  (0, 108262)	1
  (0, 10828)	2
  (0, 47214)	1
  (0, 110270)	1
  (0, 76151)	1
  (0, 97201)	1
  (0, 117759)	1
  :	:
  (0, 93353)	1
  (0, 57592)	1
  (0, 70556)	1
  (0, 39445)	1
  (0, 48550)	1
  (0, 47137)	1
  (0, 129301)	1
  (0, 24972)	1
  (0, 132081)	1
  (0, 71780)	1
  (0, 116129)	1
  (0, 119984)	1
  (0, 66798)	1
  (0, 10059)	1
  (0, 18327)	1
  (0, 116877)	1
  (0, 89133)	1
  (0, 21789)	1
  (0, 4479)	1
  (0, 124944)	1
  (0, 57189)	1
  (0, 79748)	1
  (0, 74886)	1
  (0, 35227)	1
  (0, 46201)	1


In [25]:
x_test_ct.shape

(600, 138289)

In [26]:
x_test_ct[0]

<1x138289 sparse matrix of type '<class 'numpy.int64'>'
	with 99 stored elements in Compressed Sparse Row format>

### 6. Create a dictionary to get the count of every label

In [27]:
 text=[]
 text.append(df1['label'].str.cat(sep=','))

In [28]:
text

['male,15,Student,Leo,male,15,Student,Leo,male,15,Student,Leo,male,15,Student,Leo,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,

In [29]:
def convert(lst): 
    return (lst[0].split(',')) 
  
# Driver code 
text = convert(text)
print(text)

['male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'Investment

In [30]:
dictionary = {}
for label in text :
  dictionary[label] = dictionary.get(label, 0) + 1

dictionary

{'male': 1272,
 '15': 299,
 'Student': 403,
 'Leo': 55,
 '33': 94,
 'InvestmentBanking': 70,
 'Aquarius': 286,
 'female': 728,
 '14': 74,
 'indUnk': 452,
 'Aries': 699,
 '25': 110,
 'Capricorn': 77,
 '17': 147,
 'Gemini': 21,
 '23': 93,
 'Non-Profit': 46,
 'Cancer': 76,
 'Banking': 16,
 '37': 19,
 'Sagittarius': 113,
 '26': 43,
 '24': 334,
 'Scorpio': 243,
 '27': 86,
 'Education': 118,
 '45': 14,
 'Engineering': 119,
 'Libra': 313,
 'Science': 33,
 '34': 6,
 '41': 14,
 'Communications-Media': 14,
 'BusinessServices': 21,
 'Sports-Recreation': 75,
 'Virgo': 39,
 'Taurus': 76,
 'Arts': 2,
 'Pisces': 2,
 '44': 3,
 '16': 25,
 'Internet': 20,
 'Museums-Libraries': 2,
 'Accounting': 2,
 '39': 32,
 '35': 607,
 'Technology': 607}

### 7. Transform the labels

As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_mlb=mlb.fit_transform(y_train)
y_test_mlb=mlb.transform(y_test)

In [32]:
print(y_train_mlb)
print(y_train_mlb.shape)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 1]
 [0 1 0 ... 1 0 0]]
(1400, 47)


In [33]:
print(y_test_mlb)
print(y_test_mlb.shape)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 1]]
(600, 47)


In [34]:
list(mlb.classes_)

['14',
 '15',
 '16',
 '17',
 '23',
 '24',
 '25',
 '26',
 '27',
 '33',
 '34',
 '35',
 '37',
 '39',
 '41',
 '44',
 '45',
 'Accounting',
 'Aquarius',
 'Aries',
 'Arts',
 'Banking',
 'BusinessServices',
 'Cancer',
 'Capricorn',
 'Communications-Media',
 'Education',
 'Engineering',
 'Gemini',
 'Internet',
 'InvestmentBanking',
 'Leo',
 'Libra',
 'Museums-Libraries',
 'Non-Profit',
 'Pisces',
 'Sagittarius',
 'Science',
 'Scorpio',
 'Sports-Recreation',
 'Student',
 'Taurus',
 'Technology',
 'Virgo',
 'female',
 'indUnk',
 'male']

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Just for the sake testing purposes, created a pipeline for steps 8 & 9 by choosing OneVsRestClassifier and Linear SVC, trained the model on every label and computed the accuracy scores, F1, Recall & Precision scores.

In [37]:
classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(x_train, y_train_mlb)
predicted = classifier.predict(x_test)
print(predicted)

print("Accuracy Score using OvR and Linear SVC: ",accuracy_score(y_test_mlb, predicted))
print("F1 using OvR and Linear SVC: " + str(f1_score(y_test_mlb,predicted,average='micro')))
print("Recall using OvR and Linear SVC: " + str(recall_score(y_test_mlb,predicted,average='micro')))
print("Precision using OvR and Linear SVC: " + str(average_precision_score(y_test_mlb, predicted,average='micro')))

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
Accuracy Score using OvR and Linear SVC:  0.375
F1 using OvR and Linear SVC: 0.6798378926038501
Recall using OvR and Linear SVC: 0.5591666666666667
Precision using OvR and Linear SVC: 0.5222733291183316


### 8. Choose a classifier

In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large.

In [38]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression(solver='lbfgs')
clf= OneVsRestClassifier(clf)  

### 9. Fit the classifier, make predictions and get the accuracy

In [39]:
clf.fit(x_train_ct,y_train_mlb)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [40]:
y_pred=clf.predict(x_test_ct)
y_pred

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [41]:
y_pred.shape

(600, 47)

In [42]:
y_pred_ = mlb.inverse_transform(y_pred)

In [43]:
print(y_pred_)

[('male',), ('male',), ('35', 'Technology', 'male'), ('female',), ('26', 'Libra', 'Student', 'female'), ('35', 'Aries', 'Technology', 'male'), ('Aries', 'male'), ('35', 'Aries', 'Technology', 'male'), ('Libra', 'female'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('Libra', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('Engineering', 'Libra', 'male'), ('23', '25', '33', '45', 'Aquarius', 'Banking', 'Cancer', 'Non-Profit', 'Sagittarius', 'Taurus', 'indUnk', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('Aquarius', 'male'), ('male',), ('male',), ('Libra', 'male'), ('female', 'indUnk'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('male',), ('Aquarius', 'Student', 'female'), ('female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('15', 'Lib

In [44]:
y_test_mlb.shape

(600, 47)

In [45]:
y_test_ = mlb.inverse_transform(y_test_mlb)

In [46]:
print(y_test_)

[('24', 'Engineering', 'Libra', 'male'), ('35', 'Aries', 'Technology', 'male'), ('26', 'Sagittarius', 'Science', 'female'), ('39', 'Education', 'Virgo', 'male'), ('17', 'Student', 'Virgo', 'male'), ('35', 'Aries', 'Technology', 'male'), ('24', 'Engineering', 'Libra', 'male'), ('24', 'Scorpio', 'female', 'indUnk'), ('15', 'Libra', 'Student', 'female'), ('23', 'Sagittarius', 'indUnk', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('15', 'Libra', 'Student', 'female'), ('24', 'Scorpio', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('24', 'Engineering', 'Libra', 'male'), ('15', 'Aquarius', 'Student', 'female'), ('17', 'Capricorn', 'Sports-Recreation', 'male'), ('35', 'Aries', 'Technology', 'male'), ('15', 'Aquarius', 'Student', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('17', 'Capricorn', 'Sports-Recreation', 'male'), ('24', 'Engineering', 'Libra', 'mal

In [47]:
from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [48]:
print("F1: " + str(f1_score(y_test_mlb,y_pred,average='micro')))
print("Recall: " + str(recall_score(y_test_mlb,y_pred,average='micro')))
print("Precision: " + str(average_precision_score(y_test_mlb, y_pred,average='micro')))
print("Accuracy:" + str(metrics.accuracy_score(y_test_mlb,y_pred))) 

F1: 0.6520574628682737
Recall: 0.5579166666666666
Precision: 0.47526349054996736
Accuracy:0.38166666666666665


### 10. Print true label and predicted label for any five examples


In [49]:
y_test[0:330]

674         [male, 24, Engineering, Libra]
1699         [male, 35, Technology, Aries]
1282    [female, 26, Science, Sagittarius]
1315          [male, 39, Education, Virgo]
1210            [male, 17, Student, Virgo]
                       ...                
1988         [male, 35, Technology, Aries]
428          [female, 24, indUnk, Scorpio]
1604         [male, 35, Technology, Aries]
107           [female, 17, Student, Aries]
834       [male, 17, Student, Sagittarius]
Name: labels, Length: 330, dtype: object

In [66]:
y_test[674]

['male', '24', 'Engineering', 'Libra']

In [68]:
y_pred_[107]

('15', 'Student', 'female')

In [69]:
y_pred_[428]

('17', '25', 'Non-Profit', 'Student', 'male')

In [70]:
y_pred_[60]

('35', 'Aries', 'Technology', 'male')

In [71]:
y_pred_[45]

('35', 'Aries', 'Technology', 'male')